# [Vector stores and retrievers](https://python.langchain.com/v0.2/docs/tutorials/retrievers/)
============================

本教程將使您熟悉 LangChain 的向量存儲和檢索器抽象。這些抽象旨在支持從（向量）數據庫和其他來源檢索數據，以便與 LLM 工作流程集成。它們對於需要在模型推理過程中檢索數據進行推理的應用程序非常重要，比如在檢索增強生成（RAG）的情況下（請參見我們的 RAG 教程 [這裡](https://python.langchain.com/v0.2/docs/tutorials/rag/)）。

Concepts[​](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#concepts "Direct link to Concepts")
------------------------------------------------------------------------------------------------------------

本指南專注於文本數據的檢索。我們將涵蓋以下概念：

-   文檔；
-   向量存儲；
-   檢索器。

In [ ]:
# ! pip3 install langchain langchain-chroma langchain-openai

許多使用 LangChain 開發的應用程序都包含多個步驟，其中涉及多次 LLM 調用。隨著這些應用程序變得越來越複雜，能夠檢查鏈或代理內部發生的情況變得至關重要。而檢查這些過程的最佳方法就是使用 [LangSmith](https://smith.langchain.com/)。

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

In [1]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3.1")
model

ChatOllama(model='llama3.1')

# [Documents](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#documents "Direct link to Documents")
---------------------------------------------------------------------------------------------------------------

LangChain 實現了一個 [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 抽象，旨在表示一個文本單元及其相關的元數據。它有兩個屬性：

- `page_content`：表示內容的字串；
- `metadata`：包含任意元數據的字典。

`metadata` 屬性可以捕捉有關文檔來源、與其他文檔的關係以及其他信息。需要注意的是，單個 `Document` 對象通常代表一個較大文檔的一部分。

讓我們來生成一些示例文檔：

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

在這裡，我們生成了五個文檔，這些文檔包含了元數據，指示它們來自三個不同的「來源」。

# [Vector stores​](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#vector-stores "Direct link to Vector stores")
---------------------------------------------------------------------------------------------------------------------------

向量搜索是一種常見的方法，用於存儲和搜索非結構化數據（例如非結構化文本）。其想法是將與文本相關聯的數字向量存儲起來。給定一個查詢，我們可以將其[嵌入](https://python.langchain.com/v0.2/docs/concepts/#embedding-models)為相同維度的向量，並使用向量相似度度量來識別存儲中相關的數據。

LangChain 的 [VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) 對象包含將文本和 `Document` 對象添加到存儲中的方法，並使用各種相似度度量來查詢它們。它們通常會與[嵌入](https://python.langchain.com/v0.2/docs/how_to/embed_text/)模型一起初始化，該模型決定如何將文本數據轉換為數字向量。

LangChain 包含一系列與不同向量存儲技術的[集成](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)。有些向量存儲由提供者托管（例如，各種雲提供者），使用時需要特定的憑證；有些（如 [Postgres](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pgvector/)）運行在可以本地或通過第三方運行的單獨基礎設施中；還有一些可以在內存中運行，用於輕量級工作負載。在這裡，我們將使用 [Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)，這是一個內存中實現，來演示 LangChain VectorStores 的用法。

要實例化向量存儲，我們通常需要提供一個[嵌入](https://python.langchain.com/v0.2/docs/how_to/embed_text/)模型，以指定如何將文本轉換為數字向量。在這裡，我們將使用 [OpenAI embeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/)。

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(
    model="llama3.1",
)
vectorstore = Chroma.from_documents(
    documents,
    embedding=ollama_emb,
)

調用 `.from_documents` 將會把文檔添加到向量存儲中。[VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) 實現了添加文檔的方法，這些方法也可以在對象實例化後調用。大多數實現都允許您連接到現有的向量存儲------例如，通過提供客戶端、索引名稱或其他信息。請參閱具體[集成](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)的文檔以了解更多詳情。

一旦我們實例化了一個包含文檔的 `VectorStore`，我們就可以對其進行查詢。[VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html) 包含以下查詢方法：

- 同步和異步；

- 以字符串查詢和以向量查詢；

- 返回或不返回相似度分數；

- 根據相似度和[最大邊際相關性](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search)（以平衡相似度與檢索結果的多樣性）進行查詢。

這些方法的輸出通常會包括一個 [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 對象的列表。

### [Examples](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#examples "Direct link to Examples")

根據與字符串查詢的相似度返回文檔：

In [5]:
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

非同步查詢

In [13]:
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

Return scores:

In [14]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.8116172552108765),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.9182254076004028),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.9501910209655762),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.972739815711975)]

根據與嵌入查詢的相似度返回文檔：

In [15]:
embedding = ollama_emb.embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

# [Retrievers​](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#retrievers "Direct link to Retrievers")
------------------------------------------------------------------------------------------------------------------

LangChain 的 `VectorStore` 對象不會繼承 [Runnable](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.runnables)，因此無法立即集成到 LangChain 表達式語言 [chains](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) 中。

LangChain 的 [Retrievers](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.retrievers) 是 Runnables，因此它們實現了一組標準方法（例如，同步和異步的 `invoke` 和 `batch` 操作），並設計為可以集成到 LCEL 鏈中。

我們可以自行創建一個簡單版本的檢索器，而不需要繼承 `Retriever`。如果我們選擇要用於檢索文檔的方法，我們可以輕鬆地創建一個 runnable。下面，我們將圍繞 `similarity_search` 方法構建一個簡單的檢索器：

In [6]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

向量存儲實現了一個 `as_retriever` 方法，該方法將生成一個檢索器，具體來說是一個 [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html)。這些檢索器包括特定的 `search_type` 和 `search_kwargs` 屬性，用於確定調用底層向量存儲的方法以及如何對它們進行參數設置。例如，我們可以通過以下方式來復制上述操作：

In [7]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

`VectorStoreRetriever` 支持 `"similarity"`（默認）、`"mmr"`（最大邊際相關性，如上所述）和 `"similarity_score_threshold"` 等搜索類型。我們可以使用後者來根據相似度分數篩選檢索器輸出的文檔。

檢索器可以輕鬆集成到更複雜的應用程序中，例如檢索增強生成 (RAG) 應用程序，這些應用程序將給定的問題與檢索到的上下文結合成為 LLM 的提示。下面我們展示一個簡單的範例。

In [8]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3.1")
llm

ChatOllama(model='llama3.1')

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [10]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Cats are independent pets that often enjoy their own space.


# [Learn more:​](https://python.langchain.com/v0.2/docs/tutorials/retrievers/#learn-more "Direct link to Learn more:")
--------------------------------------------------------------------------------------------------------------------

檢索策略可以是豐富而複雜的。例如：

- 我們可以從查詢中[推斷出嚴格規則和篩選條件](https://python.langchain.com/v0.2/docs/how_to/self_query/)（例如，「使用2020年後發布的文檔」）；
- 我們可以[返回與檢索到的上下文相關聯的文檔](https://python.langchain.com/v0.2/docs/how_to/parent_document_retriever/)（例如，通過某些文檔分類法）；
- 我們可以為每個上下文單位生成[多個嵌入](https://python.langchain.com/v0.2/docs/how_to/multi_vector/)；
- 我們可以從多個檢索器中[集合結果](https://python.langchain.com/v0.2/docs/how_to/ensemble_retriever/)；
- 我們可以為文檔分配權重，例如，對[最近的文檔](https://python.langchain.com/v0.2/docs/how_to/time_weighted_vectorstore/)給予更高的權重。

在操作指南的[檢索器](https://python.langchain.com/v0.2/docs/how_to/#retrievers)部分涵蓋了這些和其他內建檢索策略。

同時，也可以很容易地擴展 [BaseRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html) 類，以實現自定義檢索器。請參見我們的操作指南[這裡](https://python.langchain.com/v0.2/docs/how_to/custom_retriever/)。